# 11장. 심층 신경망 훈련

앞 장에서 인공 신경망을 소개했고 첫 번째 심층 신경망을 훈련시켰다. 하지만 두 개의 은닉층만 있는 매우 얕은 심층 신경망이었다.

고해상도 이미지에서 수백 종의 물체를 감지해야 하는 것처럼 아주 복잡한 문제를 다뤄야 한다면 어떻게 해야 할까? 아마도 수백 개의 뉴런을 가진 10개 층이 수십만 개의 연결로 이어져 있는 훨씬 더 깊은 심층 신경망을 훈련시켜야 할 것이다.

하지만 아래의 이유로 매우 어려운 일이 된다.

- 첫째, 까다로운 **그래디언트 소실 Vanishing Gradient** (또는 **그래디언트 폭주 Exploding Gradient**) 문제에 직면할 것이다. 이 현상은 심층 신경망에 영향을 주며 하위층이 훈련하기 매우 어렵게 만든다.
- 둘째, 이런 대규모 신경망에서는 훈련이 극단적으로 느려진다.
- 셋째, 수백만 개의 파라미터를 가진 모델은 훈련 세트에 과대적합될 위험이 매우 크다.

이 장에서는 이 문제들을 차례대로 살펴보고 해결 방법을 제시한다. 먼저 그래디언트 소실 문제를 설명하고 가장 널리 알려진 해결 방법을 살펴본다.

다음에는 일반 경사 하강법에 비해서 대규모 모델의 훈련 속도를 크게 높여주는 다양한 최적화 방법을 알아본다. 마지막으로 대규모 신경망을 위한 규제 기법 중 널리 알려진 몇 가지를 살펴본다.

이런 도구들을 사용하여 매우 깊은 신경망을 훈려시킬 수 있다. ***딥러닝의 세계에 온 걸 환영한다!!***

# 1. 그래디언트 소실과 폭주 문제

앞 장에서 이야기한 것처럼 역전파 알고리즘은 출력층에서 입력층으로 오차 그래디언트를 전파시키면서 진행된다. 알고리즘이 신경망의 모든 파라미터에 대한 오차 함수의 그래디언트를 계산하면 경사 하강법 단계에서 이 그래디언트를 사용하여 각 파라미터를 수정한다.

안타깝게도 알고리즘이 하위층으로 진행됨에 따라 그래디언트는 점점 작아지는 경우가 많다. 결국 경사 하강법이 하위층의 연결 가중치를 실제 변경되지 않은 채로 둔다면 훈련이 좋은 솔루션으로 수렴되지 않는다. 이 문제를 **그래디언트 소실**이라고 한다. 

반대의 경우로, 그래디언트가 점점 커져 여러 개의 층이 비정상적으로 큰 가중치로 갱신되면 알고리즘은 발산하게 되며, 이 문제를 **그래디언트 폭주**라고 한다. 이 현상은 순환 신경망에서 주로 나타난다. 

일반적으로 심층 신경망은 불안정한 그래디언트에 영향을 많이 받는데, 여러 층이 서로 다른 속도로 학습될 수 있기 때문이다.

2010년 즈음에 세이비어 글로럿과 요슈아 벤지오는 논문을 통해 몇 가지 주요한 사실을 발표했다.

- 평균이 0이고 표준편차가 1인 정규분포를 사용한 무작위 초기화 방법이 가장 인기있었다.
- 로지스틱 활성화 함수에서, 입력의 절댓값이 크면 0이나 1로 수렴하여 기울기가 0에 매우 가까워진다.
- 그래서 역전파가 될 때 사실상 신경망으로 전파시킬 그래디언트가 거의 없고 조금 있는 그래디언트는 최상위층에서부터 역전파가 진행됨에 따라 점차 약해진다.

## 1.1 세이비어 초기화와 He 초기화

글로럿과 벤지오는 이 문제를 크게 완화시키는 방법을 제안한다.

예측을 할 때는 정방향으로, 그래디언트를 역전파할 때는 역방향으로 양방향 신호가 적절하게 흘러야 한다. 신호가 죽거나 폭주 또는 소멸하지 않아야 한다.

- 그들은 적절한 신호가 흐르기 위해서는 각 층의 출력에 대한 분산이 입력에 대한 분산과 같아야 한다고 주장한다.
- 또한 역방향에서 층을 통과하기 전과 후의 그래디언트 분산이 동일해야 한다.
- 층의 입력과 출력 연결 개수가 같지 않다면 이 두 가지를 보장할 수 없다. 하지만 그들은 실전에서 매우 잘 작동한다고 입증된 대안을 제안하였다.

이는 연결 가중치를 아래 식에 기술한 방식대로 무작위로 초기화하는 것이다.

$$ \mu = 0,\space\space\sigma = \sqrt{\frac{2}{n_{inputs}+n_{outputs}}}인\space\space정규분포$$

$$ 또는 \space\space r = \sqrt{\frac{6}{n_{inputs}+n_{outputs}}}일 \space\space때 \space\space-r과 \space\space+r \space\space사이의 \space\space균등분포$$

위 식에서 $n_{inputs}$와 $n_{outputs}$는 가중치를 초기화하려는 층의 입력과 출력 연결의 개수이다. (또는 **팬인fan-in**과 **팬아웃fan-out**으로 부른다.)

이 초기화 전략을 **세이비어 초기화 Xavier Initialization** 또는 **글로럿 초기화 Glorot Initialization**라고 한다.

입력의 연결 개수가 대략 출력의 연결 개수와 비슷하면 더 간단한 공식을 사용한다. ($\sigma = \frac{1}{\sqrt{n_{inputs}}}$ 또는 $r = \frac{\sqrt{3}}{\sqrt{n_{inputs}}}$)

10장에서는 기본 세이비어 초기화 전략을 사용했다. 이를 사용하면 훈련 속도를 상당히 높일 수 있고, 현재 딥러닝의 성공을 견인한 기술 중 하나이다. 

<br>

최근 몇몇 논문은 다른 활성화 함수에 대해 비슷한 전략을 제시하는데, ReLU 활성화 함수를 위한 초기화 전략을 **He 초기화, He Initialization**이라고 부르기도 한다.

`tf.layer.dense()` 함수는 기본적으로 균등분포로 세이비어 초기화를 사용하는데, `variance_scaling_initializer()` 함수를 사용하여 He 초기화 방식으로 바꿀 수도 있다.

## 1.2 수렴하지 않는 활성화 함수

2010년 논문에서 얻은 통찰 중 하나는 활성화 함수를 잘못 선택하면 자칫 그래디언트의 소실이나 폭주로 이어질 수 있다는 점이다.

그전에는 대부분의 사람들이 생물학적 뉴런의 방식과 비슷한 시그모이드 활성화 함수가 최선의 선택일 것이라고 생각했지만, 다른 활성화 함수가 심층 신경망에서 훨씬 더 잘 작동한다는 사실이 밝혀졌다.

특히 ReLU 함수는 특정 양수값에 수렴하지 않고, 계산도 빠르다는 장점이 있다.

하지만 이조차도 완벽하지 않은데, **죽은 ReLU, Dying ReLU**라고 알려진 문제가 있다. 훈련하는 동안 일부 뉴런이 0 이외의 값을 출력하지 않는다는 의미로 죽었다고 말한다.

어떤 경우에는, 특히 큰 학습률을 사용하면 신경망의 뉴런 절반이 죽어있기도 한다. 훈련 도중 뉴런의 가중치가 바뀌어 가중치의 합이 음수가 되면 그다음부터 0을 출력하기 시작할 것이다.

ReLU 함수는 입력이 음수면 그래디언트가 0이 되기 때문에 이런 일이 생기면 뉴런이 다시 살아나기 어렵다.

이 문제를 해결하기 위해 **LeakyReLU** 같은 ReLU 함수의 변종을 사용하게 된다. 이 함수는 $LeakyReLU_a(z) = max(az, z)$로 정의되며, 하이퍼파라미터 $\alpha$가 이 함수가 새는 (leaky) 정도를 결정한다.

여기서 새는 정도란 $z<0$일 때 이 함수의 기울기이며, 일반적으로 0.01로 설정한다. 이 작은 기울기가 ReLU를 절대 죽지 않게 만들어준다.

최근 한 논문에서 여러 ReLU 함수의 변종을 비교해 얻은 결론 하나는 LeakyReLU가 ReLU보다 항상 성능이 높다는 것이다. 또한 $\alpha=0.2$가 0.01보다 더 나은 성능을 내는 것으로 보인다.

그 외 아래와 같은 함수들이 있다.

- **RReLU, Randomized leaky ReLU** : 훈련하는 동안 주어진 범위에서 $\alpha$를 무작위로 선택하고 테스트 시에는 평균을 사용한다.
- **PReLU, Parametric leaky ReLU** : $\alpha$가 훈련하는 동안 역전파에 의해 학습된다. 이 함수는 대규모 이미지 데이터셋에서 ReLU보다 성능을 크게 앞섰지만, 소규모 데이터셋에서는 훈련 세트에 과대적합될 위험이 있다.

또 하나 중요한 함수는 2015년의 한 논문에서 제안한 **ELU, Exponential Linear Unit**이라는 새로운 활성화 함수이다.

이 함수는 실험에서 다른 모든 ReLU 변종을 앞질렀으며, 훈련 시간이 줄고 신경망의 테스트 세트 성능도 더 높았다. 이 함수의 정의는 아래와 같다.

$$ ELU_a(z) = a(exp(z)-1)\space when\space z<0,\space\space z\space otherwise$$

ELU는 몇 가지를 제외하고는 ReLU와 매우 비슷하다.

- z<0일 때 음숫값이 들어오므로 활성화 함수의 평균 출력이 0에 더 가까워진다. 이는 앞서 이야기한 그래디언트 소실 문제를 완화해준다. 하이퍼파라미터 a는 z가 큰 음숫값일 때 ELU가 수렴할 값을 정의한다. 보통 1로 설정한다.
- z<0이어도 그래디언트가 0이 아니므로 죽은 뉴런을 만들지 않는다.
- a=1일 때 이 함수는 z=0에서 급격히 변동하지 않고 z=0을 포함해 모든 구간에서 매끄러워 경사 하강법의 속도를 높여준다.

ELU 활성화 함수의 주요 단점은 ReLU나 그 변종들보다 (지수 함수를 사용하기 때문에) 계산이 느리다는 점이다.

훈련하는 동안에는 수렴 속도가 빠르기 때문에 상쇄되지만 테스트 시에는 ELU 신경망이 ReLU 신경망보다 느릴 것이다.

---

**Tip.** 그럼 심층 신경망의 은닉층에는 어떤 활성화 함수를 써야 할까?

일반적으로 ELU > LeakyReLU > ReLU > tanh > 로지스틱 순서이다. 실행 속도가 중요하다면 LeakyReLU가 ELU보다 나을 수 있다.

시간과 컴퓨팅 파워가 충분하다면 교차 검증을 사용해 여러 활성화 함수를 평가해볼 수 있고, 신경망이 과대적합되었다면 RReLU, 훈련 세트가 아주 크다면 PReLU를 포함시키면 좋다.

---

텐서플로는 신경망에 사용할 수 있는 `elu()` 함수를 제공한다.  `activation` 파라미터에 `tf.nn.elu`를 써주기만 하면 된다.

## 1.3 배치 정규화

ELU(또는 다른 ReLU)와 함께 He 초기화를 사용하면 훈련 초기 단계에서 그래디언트 소실이나 폭주 문제를 크게 감소시킬 수 있지만, 훈련하는 동안 다시 발생하지 않으리란 보장은 없다.

2015년 한 논문에서 세르게이 아이오페와 크리스티안 세게디가 그래디언트 소실과 폭주 문제를 해결하기 위한 **배치 정규화, Batch Normalization, BN** 기법을 제안했다.

더 일반적으로는, 훈련하는 동안 이전 층의 파라미터가 변함에 따라 각 층에 들어오는 입력의 분포가 변화되는 문제이다. (**내부 공변량 변화 Internal Covariance Shift** 문제라고 부른다.)

이 기법은 각 층에서 활성화 함수를 통과하기 전에 모델에 연산을 하나 추가한다. 단순하게 입력 데이터의 평균을 0으로 만들고 정규화한 다음, 각 층에서 두 개의 새로운 파라미터로 결괏값의 스케일을 조정하고 이동시킨다. (하나는 스케일 조정을 위해, 다른 하나는 이동을 위해)

다시 말해 이 연산으로 모델이 층마다 입력 데이터의 최적 스케일과 평균을 학습한다.

입력 데이터의 평균을 0으로 만들고 정규화하려면 알고리즘은 평균과 표준편차를 추정해야 한다. 이를 위해 현재 미니배치에서 입력과 평균과 표준편차를 평가한다. 

테스트할 때는 평균과 표준편차를 계산할 미니배치가 없으니 전체 훈련 세트의 평균과 표준편차를 대신 사욯안다. 이 값들은 훈련 과정에서 이동 평균을 사용하여 효율적으로 계산되므로 전체적으로 보면 배치 정규화된 층마다 $\gamma$(스케일), $\beta$(이동), $\mu$(평균), $\sigma$(표준편차) 네 개의 파라미터가 학습된다.

이 기법으로 논문에서 실험했던 모든 심층 신경망의 성능이 크게 향상되었고, tanh나 심지어 로지스틱 활성화 함수 같이 수렴되는 함수를 사용하더라도 그래디언트 소실 문제가 크게 감소되었다.

또한 네트워크가 가중치 초기화에 훨씬 덜 민감해지고, 훨씬 큰 학습률을 사용할 수 있어 학습 속도를 크게 개선해주었다.

그러나 (첫 번째 은닉층에 배치 정규화를 적용하면 입력 데이터를 정규화할 필요가 없어지지만) 배치 정규화는 모델의 복잡도를 키운다. 더군다나 실행 시간 면에서도 손해이다. 층마다 추가되는 계산이 신경망의 예측이 느려지게 하기 때문이다. 

### 텐서플로를 사용해 배치 정규화 구현하기

텐서플로는 손쉽게 입력값을 중앙에 정렬하고 정규화해주는 `tf.nn.batch_normalization()` 함수를 제공하지만, 평균과 표준편차를 직접 계산해 이 함수의 매개변수로 전달해야 한다.

하지만 이는 편리한 방법이 아니므로, 대신 다음 코드처럼 이 모든 일을 처리해주는 `tf.layers.batch_normalization()` 함수를 사용하는 편이 좋다.


In [1]:
import tensorflow as tf

C:\Users\ud803\Anaconda3\envs\hands_on_ml\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ud803\Anaconda3\envs\hands_on_ml\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ud803\Anaconda3\envs\hands_on_ml\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ud803\Anaconda3\envs\hands_on_ml\lib\site-pack

In [2]:
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")

training = tf.placeholder_with_default(False, shape=(), name="training")

hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
bn1 = tf.layers.batch_normalization(hidden1, training=training, momentum=0.9)
bn1_act = tf.nn.elu(bn1)
hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
bn2 = tf.layers.batch_normalization(hidden2, training=training, momentum=0.9)
bn2_act = tf.nn.elu(bn2)
logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
logits = tf.layers.batch_normalization(logits_before_bn, training=training, momentum=0.9)

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.batch_normalization instead.


코드를 차례대로 살펴보자. 

- training placeholder 까지는 쉽게 이해할 것이다.
- `training`은 훈련하는 동안에는 True로 그 외에는 False로 설정한다. 이를 이용하여 `tf.layers.batch_normalization()` 함수가 훈련할 때 현재 미니배치의 평균과 표준편차를 사용할지 또는 (테스트할 때) 전체 훈련 세트에 대한 평균과 표준편차를 사용할지 지정한다.
- 그 다음에는 완전 연결 층과 배치 정규화 층이 번갈아 나온다.
- `tf.layers.dense()`를 사용해 완전 연결 층을 생성한다. 배치 정규화 층 이후에 활성화 함수를 적용하기 때문에 완전 연결 층에는 어떤 활성화 함수도 지정하지 않는다.
- 그리고 `training`, `momentum` 매개변수와 함께 `tf.layers.batch_normalization()` 함수를 사용해 배치 정규화 층을 만들었다.
- BN 알고리즘은 **지수 감소 Exponential Decay**를 사용해 이동 평균을 계산한다. 그래서 momentum 매개변수가 필요하다.
- 새로운 값 v가 주어지면 이동 평균 $\hat{v}$는 다음 식을 통해 갱신된다.

$$ \hat{v} \leftarrow \hat{v}\times momentum + v \times (1-momentum)$$

적절한 모멘텀 값은 일반적으로 1에 가깝다. ex) 0.9, 0.99, 0.999 (데이터셋이 크고 미니배치가 작을 경우 9를 더 넣어 1에 가깝게 한다.)

위 코드는 같은 배치 정규화 매개변수가 반복되기 때문에 코드 중복이 많다. 이런 중복을 피하기 위해 파이썬의 `functools` 모듈 안의 `partial()` 함수르 사용할 수 있다.

이 함수는 어떤 함수를 감싼 래퍼(Wrapper) 함수를 생성하며, 매개변수의 기본값을 지정할 수 있도록 도와준다.


작은 예제라서 많이 나아 보이진 않지만, 많은 층에서 같은 활성화 함수, 초기화, 규제 등을 사용한다면 훨씬 읽기 쉬운 코드가 완성된다.

구성 단계의 나머지는 10장과 같다. 비용 함수를 정의하고 옵티마이저를 생성해서 비용 함수를 최소화하도록 함수를 호출한다.

그 이후 평가 연산을 정의하고 Saver 객체를 만든다.

실행 단계도 두 가지를 제외하고는 매우 비슷하다.
- 훈련하는 동안에는 `batch_normalization()`에 의존하는 연산을 수행할 때마다 training 플레이스홀더를 True로 설정해야 한다.
- `batch_normalization()` 함수는 이동 평균을 갱신하기 위해 매 훈련 단계에서 평가할 몇 개의 연산을 만든다.

이게 전부이다. 층이 2개뿐인 이런 간단한 예제에서는 배치 정규화의 역할이 크지 않을지 모르지만 심층 신경망에서는 엄청난 차이를 만들어낼 것이다.

## 1.4 그래디언트 클리핑

그래디언트 폭주를 줄이는 쉬운 방법은 역전파될 때 일정 임곗값을 넘어서지 못하게 그래디언트를 그냥 단순히 잘라내는 것이다. 이를 **그래디언트 클리핑 Gradient Clipping** 이라고 한다.

일반적으로는 배치 정규화를 선호하지만 그래디언트 클리핑이 무엇인지 또 어떻게 구현하는지 알아두면 유용하다.

텐서플로에서 옵티마이저의 `minimize()` 함수는 그래디언트의 계산과 적용 두 가지를 모두 처리한다. 그래서 이 함수를 대신하려면 `compute_gradients()`를 먼저 호출하고 `clip_by_value()` 함수를 사용해 그래디언트를 클리핑하는 연산을 생성한다.

마지막으로 옵티마이저의 `apply_gradients()` 메서드를 사용해 클리핑된 그래디언트를 적용하는 연산을 만든다.

그러고 나서 이전과 마찬가지로 매 훈련 단계마다 `training_op`을 실행해준다. 이 코드는 그래디언트를 계산하고 그것을 -1과 1 사이로 클리핑해서 적용한다.

# 2. 미리 훈련된 층 재사용하기

일반적으로 아주 큰 규모의 DNN을 처음부터 새로 훈련시키는 것은 좋은 생각이 아니다. 해결하려는 것과 비슷한 유형의 문제를 처리한 신경망이 이미 있는지 찾아보고 그런 다음 그 신경망의 하위층을 재사용하는 것이 좋다.

이를 **전이 학습 Transfer Learning**이라고 한다. 이 방법은 훈련 속도를 크게 높여줄 뿐만 아니라 필요한 훈련 데이터도 훨씬 적다.

예를 들어 동물, 식물, 자동차 등 100개의 카테고리로 구분된 이미지를 분류하도록 훈련시킨 DNN을 가지고 있다고 가정하자.

그리고 이제 구체적인 자동차의 종류를 분류하는 DNN을 훈련시키려 한다. 이런 작업들은 비슷한 점이 많으므로 첫 번째 신경망의 일부를 재사용해봐야 한다.

---

**Note.** 만약 원래 문제에서 사용한 것과 크기가 다른 이미지를 입력으로 사용한다면 원본 모델에 맞는 크기로 변경하는 전처리 단계를 추가해야 한다.

일반적으로 전이 학습은 입력이 비슷한 저수준 특성을 가질 때 잘 작동한다.

---

## 2.1 텐서플로 모델 재사용하기

원본 모델이 텐서플로를 사용해 훈련되었다면 간단하게 바로 복원해서 새로운 작업에 훈련시킬 수 있다.

`import_meta_graph()` 함수를 사용하여 기본 그래프에 연산을 적재할 수 있다. 이 함수는 `Saver` 객체를 반환하는데, 나중에 저장된 모델 파라미터를 불러올 때 사용한다.

그다음에는 훈련 대상인 연산과 텐서를 직접 지정해야 한다. 이를 위해 그래프의 `get_operation_by_name()`과 `get_tensor_by_name()` 메서드를 사용한다.

텐서 이름은 연산 이름 뒤에 `:0`를 붙인다.

미리 훈련된 모델이 잘 문서화되어 있지 않으면 연산의 이름을 찾기 위해 그래프를 뒤져야 한다.

이런 경우 텐서보드를 사용하여 그래프를 탐색하거나 `get_operations()` 메서드를 사용해 모든 연산의 리스트를 볼 수 있다.

원본 모델을 직접 만들었다면 다른 사람들이 모델을 쉽게 재사용할 수 있도록 연산에 명확한 이름을 사용하고 문서화해놓아야 한다.

또 다른 방법은 다른 사람을 위해 중요한 연산들을 모아놓은 컬렉션을 만드는 것이다.

일반적으로 원본 모델의 일부분, 특이 하위층만 재사용하는 경우가 많다.

`import_meta_graph()` 함수를 사용해 그래프를 복원하면 원본 그래프 전체를 적재한다. 하지만 관심 대상이 아닌 층은 그냥 무시할 수 있다.

예를 들어 미리 훈련된 층 위에 새로운 층을 구축할 수 있고, 출력에 대한 손실을 계산하고 이를 최소화하기 위한 옵티마이저를 만들어야 한다.

미리 훈련된 그래프의 파이썬 코드에 접근할 수 있다면 필요한 부분만 재사용하고 나머지는 버리면 된다.

그러나 이 경우에는 훈련된 모델을 복원하는 `Saver` 객체가 필요하고 (복원하려는 변수를 지정하기 위해 필요하다. 그렇지 않으면 텐서플로는 그래프가 맞지 않는다고 에러를 내보낸다.)

새로운 모델을 저장하는 또 다른 Saver 객체가 필요하다. 예를 들어, 다음 코드는 은닉층 1,2,3만 복원한다.

---

**Tip.** 작업이 비슷할수록 (하위층부터 시작해서) 더 많은 층을 재사용하곤 한다. 매우 비슷한 작업이라면 모든 은닉층을 그대로 유지하고 출력층만 바꿀 수도 있다.

---

## 2.2 다른 프레임워크의 모델 재사용하기

만약 모델이 다른 프레임워크로 훈련되어 있다면 수동으로 모델 파라미터를 읽어 들여 적절한 변수에 할당해야 한다.

이는 매우 번거로운 작업이다. 책에는 0.5페이지 정도로 설명이 되어 있지만, 여기서는 생략한다.

## 2.3 신경망의 하위층을 학습에서 제외하기

첫 번째 DNN의 하위층은 이미지에 있는 저수준 특성을 감지하도록 학습되어서 다른 이미지 분류 작업에 유용할 것 같다.

그러므로 이 층들은 그냥 있는 그대로 재사용할 수 있다. 일반적으로 새로운 DNN을 훈련시킬 때 재사용되는 층들의 가중치를 '동결'하는 것이 좋다.

하위층의 가중치가 고정되면 (학습하려는 대상이 바뀌지 않기 때문에) 상위층의 가중치를 훈련시키기 쉽다.

*하위층의 출력은 상위층의 출력이 되므로 하위층의 가중치가 학습됨에 따라 변경되면 상위층으로 입력이 계속 바뀐다. 반대로 하위층의 가중치가 고정되면 학습이 반복되더라도 동일 이미지에 대해서는 항상 같은 출력이 상위층으로 전달된다.*

훈련하는 동안 하위층을 고정시키는 한 가지 방법은 하위층의 변수를 제외하고 훈련시킬 변수 목록을 옵티마이저에 전달하는 것이다.

첫 줄에서 은닉층 3,4와 출력층에 있는 학습할 변수 목록을 모두 구한다.

은닉층 1과 2에 있는 변수는 남겨둔다. 그다음에는 변수 목록을 옵티마이저의 `minimize()` 함수에 제공한다. 이제 은닉층 1과 2는 동결되었다!

이런 층을 **동결된 층 Frozen Layer**라고 한다.

또다른 방법은 그래프에 `stop_gradient()` 층을 추가하는 것이다.

## 2.4 동결된 층 캐싱하기

동결된 층은 변하지 않기 때문에 각 훈련 샘플에 대해 가장 위쪽의 동결된 층에서 나온 출력을 캐싱하는 것이 가능하다.

전체 데이터셋에 대한 훈련이 여러 번 반복되기 때문에 훈련 샘플마다 동결된 층을 한 번만 거친다면 (에포크마다 한 번씩이 아니라) 학습 속도를 크게 높일 수 있다.

예를 들어 (메모리가 충분하다고 가정하고) 전체 훈련 세트에 대해 하위층을 먼저 실행할 수 있다. 그리고 훈련하는 동안 훈련 샘플의 배치를 만드는 대신 은닉층 2의 출력을 배치로 만들어 훈련 연산에 주입한다.

## 2.5 상위층을 변경, 삭제, 대체하기

원본 모델의 출력층은 새로운 작업에는 거의 쓸모가 없고 심지어 새 작업을 위한 출력 뉴런 수도 같지 않을 수 있기 때문에 보통 교체된다.

비슷하게 원본 모델의 상위층은 하위층보다는 덜 유용하다. 새로운 작업에서 필요한 고수준 특성은 원본 작업에서 유용했던 특성과는 많이 다르기 때문이다. 그래서 재사용할 적절한 층의 개수를 아는 것이 중요하다.

먼저 복사한 모든 층을 동결한다. 그다음에 모델을 훈련시키고 얼마나 성능이 나오는지 지켜본다. 그 후 가장 위쪽의 은닉층 한 두 개의 동결을 해제하고 성능 향상을 확인한다.

훈련 데이터가 많을수록 많은 층을 동결 해제할 수 있다.

그래도 좋은 성능을 얻을 수 없고 훈련 데이터가 적다면 가장 위쪽의 은닉층을 제거하고 남은 은닉층을 다시 모두 동결해본다. 재사용에 적절한 층의 개수를 찾을 때까지 반복한다.

## 2.6 모델 저장소

당면한 문제와 비슷한 작업을 훈련시킨 신경망을 어디에서 찾을 수 있을까? 첫 번째로 찾아볼 곳은 자기 자신의 모델 카탈로그이다.

그러므로 자신의 모든 모델을 저장하고 나중에 쉽게 찾아볼 수 있도록 잘 정리해 놓아야 한다. 다른 선택사항은 **모델 저장소 Model Zoo**에서 찾는 것인데, 많은 사람이 다양한 문제에 대해 머신러닝 모델을 훈련시키고 있고, 친절하게도 미리 훈련된 모델을 공개하고 있다.

텐서플로는 [자체 모델 저장소](https://github.com/tensorflow/models)를 가지고 있다. 여기에는 VGG, Inception, ResNet같은 가장 성능이 뛰어난 이미지 분류 모델들이 대부분 포함되어 있다.

[카페 모델 저장소](http://goo.gl/XI02X3)도 인기이다. 여기에도 여러 데이터셋에 훈련시킨 많은 컴퓨터 비전 모델이 있다.

## 2.7 비지도 사전훈련

레이블된 훈련 데이터가 많지 않은 복잡한 문제가 있는데, 아쉽게도 비슷한 작업에 대해 훈련된 모델을 찾을 수 없을 때를 가정해보자.

이럴 때는 더 많은 레이블된 훈련 데이터를 모으는 것이 중요하다. 이것도 너무 힘이 들고 비용이 많이 든다면 **비지도 사전훈련 Unsupervised Pretraining**을 해볼 수도 있다.

즉 레이블이 없는 훈련 데이터만 많다면 **제한된 볼츠만 머신 Restricted Boltzmann Machine, RBM**이나 오토인코더 같은 비지도 특성 추출 알고리즘을 사용해 맨 하위층부터 위로 올라가면서 차례로 한 층씩 학습시킬 수 있다.

각 층은 훈련된 이전 층의 출력으로 훈련되며, 훈련 중인 층을 제외하고 다른 층은 모두 동결된다. 이런 방식으로 모든 층이 훈련되면 지도 학습으로 (즉, 역전파 알고리즘을 통해) 신경망을 세밀하게 튜닝할 수 있다.

이는 시간이 오래 걸리고 번거로운 작업이다. 하지만 잘 작동하기도 한다. 사실 제프리 힌튼의 팀은 2006년에 이 기법을 사용해 신경망의 부활과 딥러닝의 성공을 이끌었다.

2010년까지 보통 RBM을 사용한 비지도 사전훈련이 심층 신경망의 표준이었고, 그래디언트 소실 문제가 완화되고 나서야 역전파 알고리즘만을 사용하여 심층 신경망을 훈련하는 것이 널리 일반화되었다.

그러나 풀어야 할 문제가 복잡하고 재사용할 수 있는 비슷한 모델이 없으며 레이블된 훈련 데이터가 적을 때는 (오토인코더를 사용한) 비지도 사전훈련이 좋은 선택이다.

## 2.8 보조 작업으로 사전훈련

마지막 선택사항은 레이블된 훈련 데이터를 쉽게 얻거나 생성할 수 있는 보조 작업에 첫 번째 신경망을 훈련시키는 것이다.

그리고 이 신경망의 하위층을 실제 작업을 위해 재사용한다. 첫번째 신경망의 하위층은 두 번째 신경망에 재사용될 수 있는 특성 추출기를 학습하게 된다.

예를 들어 얼굴을 인식하는 시스템을 만들려고 하는데 개인별 이미지가 얼마 없다면 좋은 분류기를 훈련시키기에 충분하지 않다. 각 사람의 사진을 수백 개씩 모으기는 현실적으로 어렵기 때문이다.

그러나 인터넷에서 무작위로 많은 인물의 수집해서 두 개의 다른 이미지가 같은 사람의 것인지 감지하는 첫 번째 신경망을 훈련시킬 수 있다. 이런 신경망은 얼굴의 특성을 잘 감지하도록 학습될 것이다.

그러므로 이런 신경망의 하위층을 재사용해 적은 양의 훈련 데이터에서 얼굴을 잘 구분하는 분류기를 훈련시킬 수 있다.

종종 레이블되지 않은 훈련 샘플을 모으는 것은 비용이 저렴하지만 이를 레이블링하려면 비용이 많이 든다. 이런 상황에서 일반적인 방법은 훈련 샘플 전체를 '좋은 샘플'로 레이블하고 좋은 샘플을 오염시켜 새로운 훈련 샘플을 생성하여 '나쁜 샘플'로 레이블하는 것이다.

그런 다음 좋은 샘플과 나쁜 샘플을 분류하는 첫 번째 신경망을 훈련시킬 수 있다. 예를 들어 수백만 개의 문장을 내려받아 '좋은 샘플'이라고 레이블하고, 문장의 단어를 무작위로 바꿔 '나쁜 샘플'이라고 레이블한다. 이 하위층을 재사용하면 여러 가지 언어 처리 작업에 도움이 될 수 있다.

또 다른 접근 방식은 첫 번째 신경망이 각 훈련 샘플에 대해 점수를 출력하도록 훈련시키고 좋은 샘플의 점수가 나쁜 샘플의 점수보다 일정 마진 이상 더 크게 만드는 비용 함수를 사용하는 것이다.

이를 **최대 마진 학습 Max Margin Learning**이라고 한다.

# 3. 고속 옵티마이저

아주 큰 심층 신경망은 훈련이 심각하게 느릴 수 있다. 지금까지 훈련 속도를 네 가지 방법을 보았다.

- 여러 가중치에 좋은 초기화 전략 사용하기
- 좋은 활성화 함수 사용하기
- 배치 정규화 사용하기
- 미리 훈련된 신경망의 일부 재사용하기

훈련 속도를 크게 높일 수 있는 또다른 방법으로는 표준적인 경사 하강법 옵티마이저 대신 더 빠른 옵티마이저를 사용할 수 있다.

이 절에서는 아래의 가장 인기 있는 옵티마이저들을 소개한다. 
- **모멘텀 최적화 Momentum Opt** 
- **네스테로프 가속 경사 Nesterov Accelerated Gradient**
- **AdaGrad**
- **RMSProp**
- **Adam** 

## 3.1 모멘텀 최적화

볼링공이 매끈한 표면의 완만한 경사를 따라 굴러간다고 가정해보자. 처음에는 느리게 출발하지만 종단속도에 도달할 때까지는 빠르게 가속될 것이다.

이것이 보리스 폴락이 1964년에 제안한 **모멘텀 최적화**의 간단한 원리이다. 반대로 표준적인 경사 하강법은 경사면을 따라 일정한 크기의 스텝으로 조금씩 내려간다.

그래서 맨 아래에 도착하는 데는 시간이 더 오래 걸릴 것이다.

경사 하강법은 가중치에 대한 비용 함수 J($\theta$)의 그래디언트에 학습률을 곱한 것을 바로 차감하여 가중치 $\theta$를 갱신한다. 이 식은 이전 그래디언트가 얼마였는지 고려하지 않고, 국부적으로 그래디언트가 아주 작으면 매우 느려질 것이다.

$$ \theta \leftarrow \theta - \eta \nabla_\theta J(\theta) $$

모멘텀 최적화는 이전 그래디언트가 얼마였는지를 상당히 중요하게 생각한다. 매 반복에서 현재 그래디언트를 (학습률 $\eta$를 곱한 후) **모멘텀 벡터 Momentum Vector** m에 더하고 이 값을 빼는 방식으로 가중치를 갱신한다.

다시 말해 그래디언트를 속도가 아니라 가속도로 사용한다. 일종의 마찰저항을 표현하고 모멘텀이 너무 커지는 것을 막기 위해 이 알고리즘에는 **모멘텀 Momentum**이라는 새로운 하이퍼파라미터 $\beta$가 등장한다.

이 값은 0 (높은 마찰저항)과 1(마찰저항 없음) 사이로 설정되어야 하며, 일반적인 값은 0.9이다.

$$ m \leftarrow \beta m + \eta \nabla_\theta J(\theta)$$

$$ \theta \leftarrow \theta - m $$

모멘텀 최적화는 경사 하강법보다 더 빠르게 평편한 지역을 탈출하도록 도와준다. 경사 하강법이 가파른 경사를 꽤 빠르게 내려가지만 좁고 긴 골짜기에서는 오랜 시간이 걸린다. 반면에 모멘텀 최적화는 골짜기를 따라 바닥에 도달할 때까지 점점 더 빠르게 내려간다.

배치 정규화를 사용하지 않는 심층 신경망에서 상위층은 종종 스케일이 매우 다른 입력을 받게 된다. 모멘텀 최적화를 사용하면 이런 경우 큰 도움이 되고, **지역 최적점**을 건너뛰도록 하는 데 도움이 된다.

---

**Note.** 모멘텀 때문에 옵티마이저가 최적값에 안정되기 전까지 건너뛰었다가 다시 돌아오고, 다시 또 건너뛰는 식으로 여러 번 왔다 갔다 할 수 있다.

이것이 시스템에 마찰저항이 조금 있는 것이 좋은 이유인데, 이런 진동을 없애주고 빠르게 수렴되도록 한다.

---

텐서플로에서는 `MomentumOptimizer()` 메서드로 구현할 수 있다. 한 가지 단점은 튜닝할 하이퍼파라미터가 하나 늘어난다는 점인데, 그럼에도 실제로 모멘텀 0.9에서 보통 잘 작동하며 경사 하강법보다 거의 항상 더 빠르다.

## 3.2 네스테로프 가속 경사

**네스테로프 모멘텀 최적화 Nesterov Momentum Opt.** 또는 **네스테로프 가속 경사 Nesterov Accelerated Gradient, NAG**는 1983년 유리 네스테로프가 제안한 모멘텀 최적하의 한 변종으로, 기본 모멘텀 최적화보다 거의 항상 더 빠르다.

기본 아이디어는 현재 위치가 아니라 모멘텀의 방향으로 조금 앞서서 비용 함수의 그래디언트를 계산하는 것이다.

일반적으로 모멘텀 벡터가 올바른 방향 (즉, 최적점을 향하는 방향)을 가리킬 것이므로 이런 변경이 가능하다. 그래서 원래 위치에서의 그래디언트를 사용하는 것보다 그 방향으로 조금 더 나아가서 측정한 그래디언트를 사용하는 것이 약간 더 정확할 것이다.

이는 `MomentumOptimizer()`에서 `use_nesterov=True` 매개변수로 사용할 수 있다.

## 3.3 AdaGrad

한쪽이 길쭉한 그릇 문제를 다시 생각해보자. 경사 하강법은 가장 가파른 경사를 따라 빠르게 내려가기 시작해서 골짜기 아래로 느리게 이동한다.

알고리즘이 이를 일찍 감지하고 전역 최적점 쪽으로 좀 더 정확한 방향을 잡았더라면 좋았을 것이다. AdaGrad 알고리즘은 가장 가파른 차원을 따라 그래디언트 벡터의 스케일을 감소시켜 이 문제를 해결한다.

*(자세한 알고리즘 계산식은 생략한다)*

요약하면 이 알고리즘은 학습률을 감소시키지만 경사가 완만한 차원보다 가파른 차원에 대해 더 빠르게 감소된다.

이를 **적응적 학습률 Adaptive Learning Rate**이라고 부르며, 전역 최적점 방향으로 더 곧장 가도록 갱신되는 데 도움이 된다. 학습률 하이퍼파라미터를 덜 튜닝해도 되는 점이 또 하나의 장점이다.

AdaGrad는 간단한 2차 방정식 문제에서는 잘 작동하지만 신경망을 훈련시킬 때 너무 일찍 멈춰버리는 경향이 있다. 학습률이 너무 감소되어 전역 최적점에 도착하기 전에 알고리즘이 완전히 멈춘다.

그래서 **심층 신경망에서는 사용하지 말아야 한다.** (선형 회귀같은 간단한 작업에는 효과적일 수 있다.)

## 3.4 RMSProp

AdaGrad는 너무 빠르게 느려져서 전역 최적점에 수렴하지 못하지만 **RMSProp** 알고리즘은 가장 최근 반복에서 비롯된 그래디언트만 누적함으로써 이 문제를 해결했다.

아주 간단한 문제를 제외하고는 이 옵티마이저가 언제나 AdaGrad보다 훨씬 더 성능이 좋고, 일반적으로 모멘텀 최적화나 네스테로프 가속 경사보다 빠르게 수렴한다.

*(자세한 알고리즘 계산식은 생략한다)*

이 알고리즘은 Adam 최적화가 나오기 전까지 연구자들이 가장 선호하는 최적화 알고리즘이었다.

## 3.5 Adam 최적화

**적응적 모멘트 추정 Adaptive Momentum Estimation**을 의미하는 Adam은 모멘텀 최적화와 RMSProp의 아이디어를 합친 것이다.

모멘텀 최적화처럼 지난 그래디언트의 지수 감소 평균을 따르고 RMSProp처럼 지난 그래디언트 제곱의 지수 감소된 평균을 따른다.

*(자세한 알고리즘 계산식은 생략한다)*

이는 텐서플로의 `AdamOptimizer()` 메서드로 구현할 수 있다.

지금까지 논의한 모든 최적화 기법은 **1차 편미분 (야코비안 Jacobian)** 에만 의존했다. 최적화 이론에는 **2차 편미분 (헤시안 Hessian)** 을 기반으로 한 뛰어난 알고리즘들이 있다.

불행하게도 이런 알고리즘들은 심층 신경망에 적용하기 매우 어렵다. 하나의 출력마다 n개의 1차 편미분이 아니라 $n^2$개의 2차 편미분을 계산해야 하기 때문이다.

DNN은 전형적으로 수만 개의 파라미터를 가지기 때문에 2차 편미분 최적화 알고리즘은 메모리 용량을 넘어서는 경우가 많고 가능하다고 해도 헤시안 계산은 너무 느리다.

---

**Note. 희소 모델 훈련**

모든 최적화 알고리즘은 대부분의 파라미터가 0이 아닌 밀집(dense) 모델을 만든다. 엄청 빠르게 실행할 모델이 필요하거나 메모리를 적게 차지하는 모델이 필요하면 희소 모델을 만들 수 있다.

간단한 방법은 보통 때처럼 모델을 훈련시키고 작은 값의 가중치를 제거하는 것이다. (즉, 0으로 만든다.)

다른 방법은 훈련하는 동안 $l_1$ 규제를 강하게 적용하여 옵티마이저가 가능한 한 많은 가중치를 0으로 만들도록 강제하는 것이다. (*4절 라쏘 회귀 참조*)

하지만 어떤 경우에 이런 기법이 충분하지 않을 수 있다. 이럴 때 마지막 수단은 종종 **FTRL, Follow The Regularized Leader**라고 불리는 유리 네스테로프가 제안한 기법인 **쌍대 평균 Dual Averaging**을 적용하는 것이다.

$l_1$ 규제와 사용하면 이 기법이 매우 희소한 모델을 만들어 준다. 텐서플로는 FTRL의 변종인 `FTRL-Proximal`의 알고리즘을 구현했다.


## 3.6 학습률 스케줄링

좋은 학습률을 찾는 것이 쉬운 일은 아니다. 너무 크게 잡으면 훈련이 실제로 발산할 수 있다.

너무 작게 잡으면 최적점에 수렴하겠지만 시간이 매우 오래 걸릴 것이다. 만약 조금 높게 잡으면 처음에는 매우 빠르게 진행하겠지만 최적점 근처에서는 요동이 심해져 수렴하지 못할 것이다.

(*이는 AdaGrad, RMSProp, Adam 같은 적응적 학습률 알고리즘이 아닐 경우이다. 하지만 이 알고리즘들도 안정되려면 시간이 걸린다.*)

컴퓨팅 자원이 한정적이라면 차선의 솔루션을 만들기 위해 완전히 수렴하기 전에 훈련을 멈추어야 한다.

여러 가지 학습률에서 몇 번의 에포크만 신경망을 훈련시키고 학습 곡선을 비교해서 아주 좋은 학습률을 찾을 수도 있을지 모른다.

이상적인 학습률은 학습이 빠르고 좋은 솔루션에 수렴할 것이다.

그러나 일정한 학습률보다 더 나은 방법이 있다. 높은 학습률로 시작하고 학습 속도가 느려질 때 학습률을 낮춘다면 최적의 고정 학습률보다 좋은 솔루션에 더 빨리 도달할 수 있다.

이런 전략을 **학습 스케줄**이라고 하며 가장 보편적인 것들은 다음과 같다.

- **미리 정의된 개별적인 고정 학습률** : 예를 들어, 처음에 $\eta_0$ = 0.1로 학습을 지정하고 50 에포크 후에 0.001로 바꾼다. 이 방법이 잘 작동할 수는 있지만 적절한 학습률과 적당한 시점을 찾으려면 이리저리 바꿔봐야 한다.
- **성능 기반 스케줄링** : 매 N 스텝마다 검증 오차를 측정하고 오차가 줄어들지 않으면 $\lambda$만큼 학습률을 감소시킨다.
- **지수 기반 스케줄링** : 반복 횟수 t의 함수 $\eta(t) = \eta_0 10^{-t/r}$로 학습률을 설정한다. 이 방법은 잘 작동하지만 $\eta_0$과 r을 튜닝해야 한다.
- **거듭제곱 기반 스케줄링** : 학습률을 $\eta(t) = \eta_0(1+t/r)^{-c}$로 설정한다. 하이퍼파라미터 c는 보통 1로 지정된다. 지수 기반 스케줄링과 비슷하지만 학습률이 훨씬 느리게 감소된다.

2013년 한 논문에서 널리 알려진 학습률 스케줄링의 성능을 비교했다. 저자들은 성능 기반 스케줄링과 지수 기반 스케줄링이 둘 다 잘 작동했지만 구현과 튜닝이 쉽고 최적점에 조금 더 빨리 수렴하는 지수 기반 스케줄링이 선호된다고 결론 내렸다.

텐서플로에서의 학습 스케줄 구현은 아주 간단하다.

하이퍼파라미터 값을 지정한 후 현재 훈련 반복 횟수를 저장하되 학습되지 않는 변수 `global_step()`을 생성한다.

그다음에는 텐서플로의 `exponential_decay()` 함수를 사용해 지수 감소 학습률을 정의하고, 이 학습률로 옵티마이저를 만든다.

AdaGrad, RMSProp, Adam 최적화는 훈련하는 동안 자동으로 학습률을 감소시켜주기 때문에 학습률 스케줄링을 추가할 필요가 없다.

# 4. 과대적합을 피하기 위한 규제 방법

심층 신경망은 전형적으로 수만 개, 때로는 수백만 개의 파라미터를 가지고 있다.

파라미터가 많기 때문에 네트워크의 자유도가 매우 높으며 크고 복잡한 데이터셋을 학습할 수 있다. 하지만 이런 높은 자유도는 훈련 세트에 과대적합되기 쉽다는 것을 의미하기도 한다.

수백만 개의 파라미터가 있으면 동물원 전체를 그려낼 수 있을지 모른다. 이 절에서는 신경망에서 가장 인기 있는 규제 기법을 소개하고 텐서플로 구현 방식을 소개한다.

**조기 종료, $l_1, l_2$ 규제, 드롭아웃, 맥스-노름 규제, 데이터 증식**등에 대해 알아보자.

## 4.1 조기 종료

훈련 세트에 과대적합되는 것을 피하는 좋은 방법은 조기 종료이다. 검증 세트의 성능이 떨어지기 시작할 때 훈련을 중지시키기만 하면 된다.

텐서플로로 구현하는 한 가지 방법은 일정한 간격으로 (예를 들어 50스텝마다) 검증 세트로 모델을 평가해서 이전의 최고 성능보다 더 나을 경우 이를 최고 성능의 스냅샷으로 저장하고, 마지막 스냅샷이 저장된 이후 지난 스텝을 카운트해서 이 숫자가 어떤 한계점을 넘으면 훈련을 중지시킨다. 그리고 마지막 스냅샷을 복원한다.

실전에서 조기 종료 기법은 잘 작동하며, 다른 규제 기법과 연결하면 더 높은 성능을 얻을 수 있다.

## 4.2 $l_1$과 $l_2$ 규제

4장에서 간단한 선형 회귀에 대해 했던 것처럼 $l_1$과 $l_2$ 규제를 사용해 신경망의 연결 가중치에 제약을 가할 수 있다. (편향에는 보통 적용하지 않는다.)

텐서플로에서 이를 구현하는 방법은 비용 함수에 적절한 규제항을 추가하는 것이다. 예를 들어 가중치가 W1인 하나의 은닉층과 W2인 출력층이 있다면 다음과 같이 $l_1$규제를 적용할 수 있다.

그러나 층이 많으면 이런 방식은 매우 번거롭다. 다행히 텐서플로에는 더 나은 방법이 있는데, 변수를 생성하는 함수는 각 변수에 대한 `*_regularizer` 매개변수를 제공한다.

가중치를 매개변수로 받아 규제에 사응하는 손실을 반환하는 어떤 함수도 이 매개변수에 전달할 수 있다. 다음 코드는 이를 적용한 것이다.

이 코드는 두 개의 은닉층과 한 개의 출력층으로 이루어진 신경망을 만들고, 각 층의 가중치에 상응하는 $l_1$ 규제 손실을 계산하기 위한 노드도 그래프에 추가한다.

텐서플로는 모든 규제 손실을 포함하는 특별한 컬렉션에 이런 노드를 자동으로 추가한다. 이제 전체 손실에 다음과 같이 규제 손실을 추가해주면 된다.

**규제 손실을 추가하지 않으면 해당 변수는 그냥 무시된다!!!**

## 4.3 드롭아웃

심층 신경망에서 가장 인기 있는 규제 방법은 **드롭아웃 Dropout, 제외**이다. 이 방식은 2012년 제프리 힌튼이 제안했고, 아주 잘 작동된다고 입증되었다.

최고 성능을 내는 신경망조차도 드롭아웃을 적용해서 정확도를 1~2% 높였다. 이게 크게 느껴지지 않을 수 있지만 모델의 정확도가 95%일 때 2% 상승하는 것은 오차율이 거의 40%정도 줄어드는 것을 의미한다. (오차가 5%에서 3%로)

이 알고리즘은 매우 간단하다. 매 훈련 스텝에서 각 뉴런(입력 뉴런은 포함하고 출력 뉴런은 제외)은 임시적으로 드롭아웃될 확률 p를 가진다.

즉, 이번 훈련 스텝에서는 완전히 무시되지만 다음 스텝에서는 활성화될 수 있다. 하이퍼파라미터 p를 **드롭아웃 비율 Dropout Rate**이라고 하며 보통 50%로 지정한다.

이게 전부이다.

처음에는 이런 단순한 방식이 잘 작동된다는 것에 오히려 놀란다. 어떤 회사에서 직원들이 아침마다 출근할지 말지 동전 던지기로 결정한다면 회사의 일이 더 잘될까요?

글쎄요, 아무도 알 수 없습니다. 다만 이 회사가 이런 식으로 운영된다면 아주 사소한 일부터 아주 중요한 일까지 어떤 업무조차도 한 사람에게 전적으로 의지할 수 없게 됩니다.

이러한 전문성이 여러 사람에게 나뉘어져 있어야 한다. 이 아이디어가 회사에 실제로 맞을지는 잘 모르겠지만 확실히 신경망에는 적합하다.

즉, 개별 뉴런 하나하나가 유용해져야 하며 입력값의 작은 변화에 덜 민감해져야 한다. 결국 더 안정적인 네트워크가 되어 일반화 성능이 좋아진다.

드롭아웃을 이해하는 또 다른 방법은 각 훈련 스텝에서 고유한 네트워크가 생성된다고 생각하는 것이다. 10,000번의 훈련 스텝을 진행하면 10,000개의 다른 신경망을 훈련시키게 된다. 

결과적으로 만들어진 신경망은 이 모든 신경망을 평균한 앙상블로 볼 수 있다.

한 가지 중요한 기술적인 세부사항은, p=50%로 하면 테스트하는 동안에는 한 뉴런이 훈련 때보다 (평균 적으로) 두 배 많은 입력 뉴런과 연결된다는 점이다. 이런 점을 보상하기 위해 훈련하고 나서 각 뉴런의 연결 가중치에 0.5를 곱할 필요가 있다. 

그렇지 않으면 각 뉴런이 훈련한 것보다 거의 두 배 많은 입력 신호를 받을 것이다. 조금 더 일반적으로 말하면 훈련이 끝난 뒤 각 연결 가중치에 **보존 확률 Keep Probability**인 (1-p)를 곱해야 한다.

텐서플로에서는 `tf.layers.dropout()` 함수를 입력층이나 원하는 은닉층의 출력에 적용한다.

다음 코드는 세 개의 층으로 된 신경망에 드롭아웃 규제를 적용한다.

---

**Caution.** 보통 `tf.nn.dropout()` 함수보다 `tf.layers.dropout()` 함수를 선호한다. 

후자는 훈련하지 않을 때 작동하지 않지만 전자는 그렇지 않기 때문이다.

---

물론 앞서 배치 정규화에서 했던 것처럼 훈련할 때는 training 매개변수를 True로 지정해야 하고 테스트할 때는 False로 지정해야 한다.

모델이 과대적합되었다고 보여지면 드롭아웃 비율을 올릴 수 있다. 반대로 모델이 훈련 세트에 과소적합되면 드롭아웃 비율을 낮춰야 한다.

네트워크 층이 많을 경우에는 드롭아웃 비율을 높이고 소규모 네트워크에서는 줄이는 것이 좋다.

드롭아웃은 수렴을 상당히 느리게 만들지만 적절히 튜닝하면 매우 좋은 모델을 얻는 경우가 많다. 그래서 일반적으로 추가적인 시간과 노력을 기울일 가치가 있다.

---

**Note.** 드롭커넥트 Dropconnect는 드롭아웃의 변종으로, 전체 뉴런이 아니라 개별 연결을 무작위로 끈다.

일반적으로 드롭아웃의 성능이 더 낫다.

---

## 4.4 맥스-노름 규제

신경망에서 아주 널리 사용되는 또 다른 규제 기법은 **맥스-노름 규제 Max-Norm Regular.**이다.

이 방식은 각각의 뉴런에 대하여 연결 가중치 w가 $||w||_2 <= r$이 되도록 제한한다. 여기서 r은 맥스-노름 하이퍼파라미터이고 노름은 $l_2$ 노름을 나타낸다. 

일반적으로 매 훈련 스텝이 끝나고 $||w||_2$를 계산한 다음 w를 클리핑 한다. $w \leftarrow w \frac{r}{||w||_2}$

r을 줄이면 규제의 정도가 커져 과대적합을 감소시키는데, 맥스-노름 규제는 배치 정규화를 사용하지 않았을 때 그래디언트 감소/폭주 문제를 완화하는 데 도움을 줄 수 있다.

텐서플로에는 내장되어 있는 기능은 아니지만 구현이 어렵지는 않다.

다음 코드는 첫 번째 은닉층의 가 중치를 구한 다음 `clip_by_norm()` 함수를 사용해 두 번째 축을 따라 각 행 벡터의 최대 노름이 1.0이 되도록 가중치를 클리핑하는 연산을 만든다.


그런 다음 이 연산을 매 훈련 스텝이 끝난 후 실행해주면 된다.

일반적으로 이 클리핑을 모든 은닉층에 적용해야 하는데, 문제는 없지만 조금 번잡하다. 

더 깔끔하게 하려면 `max_norm_regularizer()` 함수를 만들어 앞서 보았던 `l1_regularizer()` 함수와 같이 사용하는 것이다.

맥스-노름 규제는 전체 손실 함수에 규제 손실 항을 추가할 필요가 없다. 하지만 그렇더라도 각 훈련 스텝이 끝나고 `clip_weights` 연산을 실행시켜야 하므로 이에 대한 핸들을 얻을 수 있어야 한다.

따라서 맥스-노름을 클리핑하는 `clip_weights` 연산을 max_norm 컬렉션에 추가해야 한다. 이 클리핑 연산을 추출해 매 훈련 스텝이 끝난 후에 실행시키면 된다.

## 4.5 데이터 증식

마지막 규제 기법인 **데이터 증식 Data Augmentation**은 기존의 데이터에서 새로운 데이터를 생성해 인공적으로 훈련 세트의 크기를 늘린다.

이 방식은 과대적합을 줄이므로 규제의 방도로 사용된다. 이 기법은 실제와 같은 훈련 샘플을 생성하는 것으로, 이상적으로는 사람이 인공적으로 만든 샘플인지 아닌지 구분할 수 없어야 한다.

무엇보다 단순히 **백색 소음 White Noise**을 추가하는 것은 도움이 되지 않고, 적용한 수정사항이 학습 가능한 것이어야 한다.

예를 들어 버섯 이미지를 구분하는 모델이 있다면 훈련 세트에 있는 모든 이미지를 다양하게 조금씩 이동, 회전하거나 크기를 바꿔서 만든 이미지를 훈련 세트에 추가한다.

이렇게 하면 모델이 사진에 있는 버섯의 위치, 각도, 크기에 덜 민감해진다. 빛에 대해서도 모델이 민감하지 않게 하려면 명암을 달리하여 여러 이미지를 생성한다.

이런 변환들을 결합하면 훈련 세트의 크기를 크게 늘릴 수 있다.

훈련하는 동안 동적으로 샘플을 생성하는 것이 저장 공간이나 네트워크 대역폭을 낭비하지 않기 때문에 선호되는 방법이다. 

---

**Note.** 아주 깊은 심층 신경망을 훈련시킬 수 있는 또 다른 강력한 기법은 **스킵 커넥션 Skip Connection**을 추가하는 것이다. 

이는 한 층의 입력을 상위층의 출력으로 보내는 것을 말하는데, 13장에서 자세히 살펴보겠다.

---

## 5. 실용적 가이드라인

이 장에서 많은 종류의 기법을 다루어 어떤 것을 써야 할지 궁금할 수 있다.

보통 아래의 설정이 대부분의 경우에 잘 맞는다.

- 초기화 : He 초기화
- 활성화 함수 : ELU
- 정규화 : 배치 정규화
- 규제 : 드롭아웃
- 옵티마이저 : 네스테로프 가속 경사
- 학습률 스케줄링 : 없음

물론 비슷한 문제를 해결한 모델을 찾았다면 미리 학습된 신경망의 일부를 재사용해야 한다.

다음과 같은 경우에는 위 기본 설정을 바꿀 필요가 있다.

- 좋은 학습률을 찾을 수 없다면 지수 감소 같은 학습 스케줄을 추가해볼 수 있다.
- 훈련 세트가 너무 작다면 데이터 증식을 수행할 수 있다.
- 희소 모델이 필요하면 $l_1$ 규제를 추가한다. 또는 훈련이 끝난 뒤 작은 가중치를 0으로 만든다. 
- 실행 속도가 아주 빠른 모델을 필요로 하면 배치 정규화를 빼고 ELU 함수를 LeakyReLU로 바꾼다. 희소 모델을 만드는 것도 도움이 된다.

이제 매우 깊은 신경망을 훈련시킬 준비가 되었지만, 인내심이 많아야 할 것이다.

훈련이 끝날 때까지 며칠 혹은 심지어 몇 달을 기다려야 할지 모르기 때문이다. 다음 장에서는 텐서플로를 사용해 연산을 여러 대의 서버와 GPU에 분산시켜 병렬로 실행시키는 방법을 알아보겠다.